# Working with missing data

In this section, we will discuss missing (also referred to as NA) values in pandas.

**Note**

The choice of using **NaN** internally to denote missing data was largely for simplicity and performance reasons. Starting from pandas 1.0, some optional data types start experimenting with a native **NA** scalar using a mask-based approach. See here for more.

See the cookbook for some advanced strategies.

# Values considered “missing”

As data comes in many shapes and forms, pandas aims to be flexible with regard to handling missing data. While **NaN** is the default missing value marker for reasons of computational speed and convenience, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object. In many cases, however, the Python **None** will arise and we wish to also consider that “missing” or “not available” or “NA”.

**Note**

If you want to consider inf and -inf to be “NA” in computations, you can set 

**pandas.options.mode.use_inf_as_na = True.**


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(
    np.random.randn(5, 3),
    index=["a", "c", "e", "f", "h"],
    columns=["one", "two", "three"],
)


In [3]:
df["four"] = "bar"

In [6]:
df["five"] = df["one"] > 0

In [7]:
df

,one,two,three,four,five
a,1.759126,-0.713632,-1.183805,bar,True
c,0.810318,1.026181,0.025248,bar,True
e,0.490947,0.138604,0.823038,bar,True
f,-0.547248,1.349807,0.108538,bar,False
h,-0.060108,-0.379824,-1.274125,bar,False


In [6]:
df2 = df.reindex(["a", "b", "c", "d", "e", "f", "g", "h"])
df2

,one,two,three,four,five
a,-0.428489,-1.137496,1.494745,bar,False
b,NaN,NaN,NaN,NaN,NaN
c,1.537751,-0.708188,0.796904,bar,True
d,NaN,NaN,NaN,NaN,NaN
e,0.438832,0.376937,1.076602,bar,True
f,1.607654,0.615904,-1.848926,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,0.939784,-1.314244,-1.167946,bar,True


To make detecting missing values easier (and across different array dtypes), pandas provides the isna() and notna() functions, which are also methods on Series and DataFrame objects:

In [7]:
df2["one"]

a   -0.428489
b         NaN
c    1.537751
d         NaN
e    0.438832
f    1.607654
g         NaN
h    0.939784
Name: one, dtype: float64

In [8]:
pd.isna(df2["one"])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [9]:
df2["four"].notna()

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: four, dtype: bool

In [10]:
df2.isna()

,one,two,three,four,five
a,False,False,False,False,False
b,True,True,True,True,True
c,False,False,False,False,False
d,True,True,True,True,True
e,False,False,False,False,False
f,False,False,False,False,False
g,True,True,True,True,True
h,False,False,False,False,False


**Warning**

One has to be mindful that in Python (and NumPy), the **nan's** don’t compare equal, but None's **do**. Note that pandas/NumPy uses the fact that **np.nan != np.nan,** and treats **None** like **np.nan**.

In [11]:
None == None  # noqa: E711

True

In [12]:
np.nan == np.nan

False

So as compared to above, a scalar equality comparison versus a None/np.nan doesn’t provide useful information.

In [13]:
df2["one"] == np.nan

a    False
b    False
c    False
d    False
e    False
f    False
g    False
h    False
Name: one, dtype: bool

### Integer dtypes and missing data


Because **NaN** is a float, a column of integers with even one missing values is cast to floating-point dtype (see Support for integer NA for more). pandas provides a nullable integer array, which can be used by explicitly requesting the dtype:

In [14]:
pd.Series([1, 2, np.nan, 4], dtype=pd.Int64Dtype())

0       1
1       2
2    <NA>
3       4
dtype: Int64

Alternatively, the string alias **dtype='Int64'** (note the capital "I") can be used.

See Nullable integer data type for more.

## Datetimes

For datetime64[ns] types, NaT represents missing values. This is a pseudo-native sentinel value that can be represented by NumPy in a singular dtype (datetime64[ns]). pandas objects provide compatibility between NaT and NaN.

In [15]:
df2 = df.copy()
df2["timestamp"] = pd.Timestamp("20120101")
df2

,one,two,three,four,five,timestamp
a,-0.428489,-1.137496,1.494745,bar,False,2012-01-01
c,1.537751,-0.708188,0.796904,bar,True,2012-01-01
e,0.438832,0.376937,1.076602,bar,True,2012-01-01
f,1.607654,0.615904,-1.848926,bar,True,2012-01-01
h,0.939784,-1.314244,-1.167946,bar,True,2012-01-01


In [16]:
df2.loc[["a", "c", "h"], ["one", "timestamp"]] = np.nan
df2


,one,two,three,four,five,timestamp
a,NaN,-1.137496,1.494745,bar,False,NaT
c,NaN,-0.708188,0.796904,bar,True,NaT
e,0.438832,0.376937,1.076602,bar,True,2012-01-01
f,1.607654,0.615904,-1.848926,bar,True,2012-01-01
h,NaN,-1.314244,-1.167946,bar,True,NaT


In [17]:
df2.dtypes.value_counts()

float64           3
object            1
datetime64[ns]    1
bool              1
dtype: int64

### Inserting missing data
You can insert missing values by simply assigning to containers. The actual missing value used will be chosen based on the dtype.

For example, numeric containers will always use **NaN**  regardless of the missing value type chosen:

In [18]:
s = pd.Series([1, 2, 3])
s.loc[0] = None
s


0    NaN
1    2.0
2    3.0
dtype: float64

Likewise, datetime containers will always use NaT.

For object containers, pandas will use the value given:

In [19]:
s = pd.Series(["a", "b", "c"])
s.loc[0] = None
s.loc[1] = np.nan
s


0    None
1     NaN
2       c
dtype: object

## Calculations with missing data


Missing values propagate naturally through arithmetic operations between pandas objects.

In [20]:
a = pd.DataFrame(
    np.random.randn(5, 3),
    index=["a", "c", "e", "f", "h"],
    columns=["one", "two", "three"],
)
a

,one,two,three
a,0.414318,-0.385738,-1.136747
c,-0.856698,0.922684,-0.308979
e,-0.664644,-0.054859,0.838842
f,-0.112066,-0.479347,-0.518381
h,0.825911,-1.264201,-0.495070


In [21]:
b = pd.DataFrame(
    np.random.randn(5, 3),
    index=["a", "c", "e", "f", "h"],
    columns=["one", "two", "three"],
)
b

,one,two,three
a,-1.056650,-1.339990,0.406410
c,-0.940589,-0.195771,0.238130
e,-0.060251,-0.729108,-0.476921
f,-1.092709,-1.367877,0.379848
h,1.403223,-0.041413,0.561941


In [22]:
a+b

,one,two,three
a,-0.642332,-1.725728,-0.730337
c,-1.797287,0.726913,-0.070849
e,-0.724895,-0.783967,0.361921
f,-1.204775,-1.847224,-0.138533
h,2.229134,-1.305614,0.066871


The descriptive statistics and computational methods discussed in the data structure overview (and listed here and here) are all written to account for missing data. For example:

- When summing data, NA (missing) values will be treated as zero.

- If the data are all NA, the result will be 0.

- Cumulative methods like cumsum() and cumprod() ignore NA values by default, but preserve them in the resulting arrays. To override this behaviour and include NA values, use skipna=False.

In [23]:
df

,one,two,three,four,five
a,-0.428489,-1.137496,1.494745,bar,False
c,1.537751,-0.708188,0.796904,bar,True
e,0.438832,0.376937,1.076602,bar,True
f,1.607654,0.615904,-1.848926,bar,True
h,0.939784,-1.314244,-1.167946,bar,True


In [24]:
df["one"].sum()

4.095531549346713

In [25]:
df.mean(1)

a   -0.017810
c    0.656617
e    0.723093
f    0.343658
h   -0.135601
dtype: float64

In [26]:
df.cumsum()

,one,two,three,four,five
a,-0.428489,-1.137496,1.494745,bar,0
c,1.109262,-1.845683,2.291649,barbar,1
e,1.548093,-1.468746,3.368251,barbarbar,2
f,3.155747,-0.852843,1.519324,barbarbarbar,3
h,4.095532,-2.167086,0.351378,barbarbarbarbar,4


In [27]:
df.cumsum(skipna=False)

,one,two,three,four,five
a,-0.428489,-1.137496,1.494745,bar,0
c,1.109262,-1.845683,2.291649,barbar,1
e,1.548093,-1.468746,3.368251,barbarbar,2
f,3.155747,-0.852843,1.519324,barbarbarbar,3
h,4.095532,-2.167086,0.351378,barbarbarbarbar,4


# Sum/prod of empties/nans

**Warning**

This behavior is now standard as of v0.22.0 and is consistent with the default in numpy; previously sum/prod of all-NA or empty Series/DataFrames would return NaN. See v0.22.0 whatsnew for more.

The sum of an empty or all-NA Series or column of a DataFrame is 0.

In [28]:
pd.Series([np.nan]).sum()

0.0

In [29]:
pd.Series([], dtype="float64").sum()

0.0

The product of an empty or all-NA Series or column of a DataFrame is 1.

In [30]:
pd.Series([np.nan]).prod()

1.0

In [31]:
pd.Series([], dtype="float64").prod()

1.0

# NA values in GroupBy


NA groups in GroupBy are automatically excluded. This behavior is consistent with R, for example:

In [32]:
df

,one,two,three,four,five
a,-0.428489,-1.137496,1.494745,bar,False
c,1.537751,-0.708188,0.796904,bar,True
e,0.438832,0.376937,1.076602,bar,True
f,1.607654,0.615904,-1.848926,bar,True
h,0.939784,-1.314244,-1.167946,bar,True


In [33]:
df.groupby("one").mean()

,two,three,five
one,,,
-0.428489,-1.137496,1.494745,False
0.438832,0.376937,1.076602,True
0.939784,-1.314244,-1.167946,True
1.537751,-0.708188,0.796904,True
1.607654,0.615904,-1.848926,True


See the groupby section here for more information.

## Cleaning / filling missing data
pandas objects are equipped with various data manipulation methods for dealing with missing data.

# Filling missing values: fillna

fillna() can “fill in” NA values with non-NA data in a couple of ways, which we illustrate:

**Replace NA with a scalar value**

In [34]:
df2

,one,two,three,four,five,timestamp
a,NaN,-1.137496,1.494745,bar,False,NaT
c,NaN,-0.708188,0.796904,bar,True,NaT
e,0.438832,0.376937,1.076602,bar,True,2012-01-01
f,1.607654,0.615904,-1.848926,bar,True,2012-01-01
h,NaN,-1.314244,-1.167946,bar,True,NaT


In [35]:
df2.fillna(0)

,one,two,three,four,five,timestamp
a,0.000000,-1.137496,1.494745,bar,False,0
c,0.000000,-0.708188,0.796904,bar,True,0
e,0.438832,0.376937,1.076602,bar,True,2012-01-01 00:00:00
f,1.607654,0.615904,-1.848926,bar,True,2012-01-01 00:00:00
h,0.000000,-1.314244,-1.167946,bar,True,0


In [36]:
df2["one"].fillna("missing")

a     missing
c     missing
e    0.438832
f    1.607654
h     missing
Name: one, dtype: object

df

df.fillna(method="pad")

df

df.fillna(method="pad", limit=1)

### Filling with a PandasObject
You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series must match the columns of the frame you wish to fill. The use case of this is to fill a DataFrame with the mean of that column.

In [41]:
df = pd.DataFrame(np.random.randn(10000, 4))
df.iloc[:9998] = np.nan
sdf = df.astype(pd.SparseDtype("float", np.nan))
sdf.head()

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [42]:
sdf.dtypes

0    Sparse[float64, nan]
1    Sparse[float64, nan]
2    Sparse[float64, nan]
3    Sparse[float64, nan]
dtype: object

In [43]:
sdf.sparse.density

0.0002

As you can see, the density (% of values that have not been “compressed”) is extremely low. This sparse object takes up much less memory on disk (pickled) and in the Python interpreter.

In [44]:
'dense : {:0.2f} bytes'.format(df.memory_usage().sum() / 1e3)

'dense : 320.13 bytes'

In [45]:
'sparse: {:0.2f} bytes'.format(sdf.memory_usage().sum() / 1e3)

'sparse: 0.22 bytes'

Functionally, their behavior should be nearly identical to their dense counterparts.

# SparseArray
arrays.SparseArray is a ExtensionArray for storing an array of sparse values (see dtypes for more on extension arrays). It is a 1-dimensional ndarray-like object storing only values distinct from the fill_value:

In [46]:
arr = np.random.randn(10)


In [47]:
arr[2:5] = np.nan

In [48]:
arr[7:8] = np.nan

In [49]:
sparr = pd.arrays.SparseArray(arr)

In [50]:
sparr

[0.49435150453632515, 0.02362110805436182, nan, nan, nan, -0.0949576280498383, 1.0399881794396344, nan, 0.7184644493356547, 0.23660609584795705]
Fill: nan
IntIndex
Indices: array([0, 1, 5, 6, 8, 9])

A sparse array can be converted to a regular (dense) ndarray with numpy.asarray()

In [51]:
np.asarray(sparr)

array([ 0.4943515 ,  0.02362111,         nan,         nan,         nan,
       -0.09495763,  1.03998818,         nan,  0.71846445,  0.2366061 ])

# SparseDtype
The SparseArray.dtype property stores two pieces of information

1. The dtype of the non-sparse values

2. The scalar fill value

In [52]:
sparr.dtype

Sparse[float64, nan]

A SparseDtype may be constructed by passing only a dtype

In [53]:
pd.SparseDtype(np.dtype('datetime64[ns]'))

Sparse[datetime64[ns], NaT]

in which case a default fill value will be used (for NumPy dtypes this is often the “missing” value for that dtype). To override this default an explicit fill value may be passed instead

In [54]:
pd.SparseDtype(np.dtype('datetime64[ns]'),
               fill_value=pd.Timestamp('2017-01-01'))

Sparse[datetime64[ns], Timestamp('2017-01-01 00:00:00')]

Finally, the string alias 'Sparse[dtype]' may be used to specify a sparse dtype in many places

In [55]:
pd.array([1, 0, 0, 2], dtype='Sparse[int]')

[1, 0, 0, 2]
Fill: 0
IntIndex
Indices: array([0, 3])

# Sparse accessor
pandas provides a .sparse accessor, similar to .str for string data, .cat for categorical data, and .dt for datetime-like data. This namespace provides attributes and methods that are specific to sparse data.

In [56]:
s = pd.Series([0, 0, 1, 2], dtype="Sparse[int]")

In [57]:
s.sparse.density

0.5

In [58]:
s.sparse.fill_value

0

This accessor is available only on data with SparseDtype, and on the Series class itself for creating a Series with sparse data from a scipy COO matrix with.

## New in version 0.25.0.

A .sparse accessor has been added for DataFrame as well. See Sparse accessor for more.

# Sparse calculation
You can apply NumPy ufuncs to **SparseArray** and get a **SparseArray** as a result.

In [59]:
arr = pd.arrays.SparseArray([1., np.nan, np.nan, -2., np.nan])

np.abs(arr)

[1.0, nan, nan, 2.0, nan]
Fill: nan
IntIndex
Indices: array([0, 3])

The ufunc is also applied to **fill_value**. This is needed to get the correct dense result.

In [60]:
arr = pd.arrays.SparseArray([1., -1, -1, -2., -1], fill_value=-1)
np.abs(arr)

[1.0, 1, 1, 2.0, 1]
Fill: 1
IntIndex
Indices: array([0, 3])

In [61]:
np.abs(arr).to_dense()

array([1., 1., 1., 2., 1.])

# Migrating

**Note**

SparseSeries and SparseDataFrame were removed in pandas 1.0.0. This migration guide is present to aid in migrating from previous versions.

In older versions of pandas, the SparseSeries and SparseDataFrame classes (documented below) were the preferred way to work with sparse data. With the advent of extension arrays, these subclasses are no longer needed. Their purpose is better served by using a regular Series or DataFrame with sparse values instead.

**Note**

There’s no performance or memory penalty to using a Series or DataFrame with sparse values, rather than a SparseSeries or SparseDataFrame.

This section provides some guidance on migrating your code to the new style. As a reminder, you can use the Python warnings module to control warnings. But we recommend modifying your code, rather than ignoring the warning.

**Construction**

From an array-like, use the regular Series or DataFrame constructors with SparseArray values.

In [62]:
pd.DataFrame({"A": pd.arrays.SparseArray([0, 1])})

,A
0,0
1,1


From a SciPy sparse matrix, use DataFrame.sparse.from_spmatrix(),

In [63]:
# New way
from scipy import sparse

mat = sparse.eye(3)

df = pd.DataFrame.sparse.from_spmatrix(mat, columns=['A', 'B', 'C'])

df.dtypes

A    Sparse[float64, 0]
B    Sparse[float64, 0]
C    Sparse[float64, 0]
dtype: object

**Conversion**

From sparse to dense, use the .sparse accessors

In [64]:
df.sparse.to_dense()

,A,B,C
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0


In [65]:
df.sparse.to_coo()

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in COOrdinate format>

From dense to sparse, use DataFrame.astype() with a SparseDtype.

In [66]:
dense = pd.DataFrame({"A": [1, 0, 0, 1]})


In [67]:
dtype = pd.SparseDtype(int, fill_value=0)
dense.astype(dtype)

,A
0,1
1,0
2,0
3,1


##### Sparse Properties

Sparse-specific properties, like density, are available on the .sparse accessor.

In [68]:
df.sparse.density

0.3333333333333333

#### General differences

In a SparseDataFrame, all columns were sparse. A DataFrame can have a mixture of sparse and dense columns. As a consequence, assigning new columns to a DataFrame with sparse values will not automatically convert the input to be sparse.

Instead, you’ll need to ensure that the values being assigned are sparse

In [69]:
df = pd.DataFrame({"A": pd.arrays.SparseArray([0, 1])})

In [70]:
df['B'] = [0, 0]  # remains dense

In [71]:
df['B'].dtype

dtype('int64')

In [72]:
df['B'] = pd.arrays.SparseArray([0, 0])

In [73]:
df['B'].dtype

Sparse[int64, 0]

The SparseDataFrame.default_kind and SparseDataFrame.default_fill_value attributes have no replacement.

# Interaction with scipy.sparse
Use DataFrame.sparse.from_spmatrix() to create a DataFrame with sparse values from a sparse matrix.

#### New in version 0.25.0.

In [74]:
from scipy.sparse import csr_matrix

arr = np.random.random(size=(1000, 5))

arr[arr < .9] = 0

sp_arr = csr_matrix(arr)

sp_arr

<1000x5 sparse matrix of type '<class 'numpy.float64'>'
	with 479 stored elements in Compressed Sparse Row format>

In [75]:
sdf = pd.DataFrame.sparse.from_spmatrix(sp_arr)
sdf.head()

,0,1,2,3,4
0,0.978601,0.990133,0.0,0.000000,0.0
1,0.000000,0.000000,0.0,0.000000,0.0
2,0.000000,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.0,0.911242,0.0
4,0.000000,0.000000,0.0,0.000000,0.0


In [76]:

sdf.dtypes

0    Sparse[float64, 0]
1    Sparse[float64, 0]
2    Sparse[float64, 0]
3    Sparse[float64, 0]
4    Sparse[float64, 0]
dtype: object

All sparse formats are supported, but matrices that are not in COOrdinate format will be converted, copying data as needed. To convert back to sparse SciPy matrix in COO format, you can use the DataFrame.sparse.to_coo() method:

In [77]:
sdf.sparse.to_coo()

<1000x5 sparse matrix of type '<class 'numpy.float64'>'
	with 479 stored elements in COOrdinate format>

meth:Series.sparse.to_coo is implemented for transforming a Series with sparse values indexed by a MultiIndex to a scipy.sparse.coo_matrix.

The method requires a MultiIndex with two or more levels.

In [78]:
s = pd.Series([3.0, np.nan, 1.0, 3.0, np.nan, np.nan])
s.index = pd.MultiIndex.from_tuples(
    [
        (1, 2, "a", 0),
        (1, 2, "a", 1),
        (1, 1, "b", 0),
        (1, 1, "b", 1),
        (2, 1, "b", 0),
        (2, 1, "b", 1),
    ],
    names=["A", "B", "C", "D"],
)


ss = s.astype('Sparse')
ss

A  B  C  D
1  2  a  0    3.0
         1    NaN
   1  b  0    1.0
         1    3.0
2  1  b  0    NaN
         1    NaN
dtype: Sparse[float64, nan]

In the example below, we transform the Series to a sparse representation of a 2-d array by specifying that the first and second MultiIndex levels define labels for the rows and the third and fourth levels define labels for the columns. We also specify that the column and row labels should be sorted in the final sparse representation.

In [79]:
A, rows, columns = ss.sparse.to_coo(
    row_levels=["A", "B"], column_levels=["C", "D"], sort_labels=True
)


A

<3x4 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in COOrdinate format>

In [80]:
A.todense()

matrix([[0., 0., 1., 3.],
        [3., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [81]:
rows

[(1, 1), (1, 2), (2, 1)]

In [82]:
columns

[('a', 0), ('a', 1), ('b', 0), ('b', 1)]

Specifying different row and column labels (and not sorting them) yields a different sparse matrix:

In [83]:
A, rows, columns = ss.sparse.to_coo(
    row_levels=["A", "B", "C"], column_levels=["D"], sort_labels=False
)


A

<3x2 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in COOrdinate format>

In [84]:
A.todense()

matrix([[3., 0.],
        [1., 3.],
        [0., 0.]])

In [85]:
rows

[(1, 2, 'a'), (1, 1, 'b'), (2, 1, 'b')]

In [86]:
columns

[0, 1]

A convenience method Series.sparse.from_coo() is implemented for creating a Series with sparse values from a scipy.sparse.coo_matrix.

In [87]:
from scipy import sparse

A = sparse.coo_matrix(([3.0, 1.0, 2.0], ([1, 0, 0], [0, 2, 3])), shape=(3, 4))

A

<3x4 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in COOrdinate format>

In [88]:
A.todense()

matrix([[0., 0., 1., 2.],
        [3., 0., 0., 0.],
        [0., 0., 0., 0.]])

The default behaviour (with dense_index=False) simply returns a Series containing only the non-null entries.

In [89]:
ss = pd.Series.sparse.from_coo(A)

Specifying dense_index=True will result in an index that is the Cartesian product of the row and columns coordinates of the matrix. Note that this will consume a significant amount of memory (relative to dense_index=False) if the sparse matrix is large (and sparse) enough.

In [90]:
ss_dense = pd.Series.sparse.from_coo(A, dense_index=True)
ss_dense

0  0    NaN
   1    NaN
   2    1.0
   3    2.0
1  0    3.0
   1    NaN
   2    NaN
   3    NaN
2  0    NaN
   1    NaN
   2    NaN
   3    NaN
dtype: Sparse[float64, nan]

### Reference: https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html